In [32]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Tweets').getOrCreate()
filename = 'gs://bigdata-general/clean_text/clean_text_tweets_part_0.csv'
#df = spark.read.option("delimiter", ";").csv("gs://bigdata-general/tweets.csv")
spark_csv = spark.read.options(header='True', delimiter=',', inferSchema='True', multiline='True').csv(filename)
spark_csv.take(5)

[Row(_c0=0, replies=19, likes=267, retweets=155, text='Central Banking Debt Serfdom . This is as simple as it . the money is the world . And now we have .', Date='2017-01-27', Category='2.0'),
 Row(_c0=1, replies=3, likes=5, retweets=3, text='How China - Probably what today', Date='2017-09-08', Category='1.0'),
 Row(_c0=2, replies=19, likes=1248, retweets=434, text="I would not be interested in if ' t want to ban it .", Date='2017-09-14', Category='0.0'),
 Row(_c0=3, replies=12, likes=509, retweets=238, text='1 / While there are no a about the path a monetary good will take as it is , a curious pattern during the relatively brief history of s monetization .', Date='2017-11-28', Category='2.0'),
 Row(_c0=4, replies=5, likes=135, retweets=110, text='How many days it took to get from : $ 0 - $ 1000 : 1789 days $ 1000 - $ 2000 : 1271 days $ 2000 - $ 3000 : 23 days $ 3000 - $ 4000 : 62 days $ 4000 - $ 5000 : 61 days $ 5000 - $ 6000 : 8 days $ 6000 - $ 7000 : 13 days $ 7000 - $ 8000 : 14 day

In [45]:
from pyspark.sql.functions import col
spark_csv = spark_csv.withColumn("Category", col("Category").cast("int"))

In [52]:
# Drop rows with null values due to errors reading the csv.
spark_csv = spark_csv.na.drop()
# Drop Category 30?
spark_csv = spark_csv.filter(spark_csv.Category != 30)

In [114]:
# Create 
# c0 = spark_csv.filter(spark_csv.Category == 0)
# text_cat = []
# for each in c0.select('text').collect():
#     text_cat.append(each[0])

# print(text_cat)
text_cat = []
for category in range(4): # [0,1,2,3]
    local_cat = []
    print(category)
    cat = spark_csv.filter(spark_csv.Category == category)
    for each in cat.select('text').collect():
        local_cat.append(each[0])
    local_cat = " ".join(local_cat)
    text_cat.append(local_cat)

0
1
2
3


In [ ]:
text_cat[1]

In [122]:
from pyspark.sql.types import StringType
grouped_df = spark.createDataFrame(text_cat, StringType())
grouped_df.show()


+--------------------+
|               value|
+--------------------+
|I would not be in...|
|How China - Proba...|
|Central Banking D...|
|Buckle in . Stop ...|
+--------------------+



In [ ]:
# c0 = spark_csv.filter(spark_csv.Category == 1)
# text_cat = []
# for each in c0.select('text').collect():
#     text_cat.append(each[0])

# #print(text_cat)
# text_cat = " ".join(text_cat)
# print(text_cat)

In [121]:

spark_csv.groupBy("Category").count().show()

+--------+-----+
|Category|count|
+--------+-----+
|       1|  221|
|       3|24311|
|       4| 1029|
|       2| 2156|
|       0|    1|
+--------+-----+



In [123]:
# Convert spark_csv to RDD
spark_rdd = grouped_df.rdd

In [ ]:
# Extract 'text' column from a Spark RDD
#spark_rdd_text = spark_rdd.map(lambda row: row['text'])
tweets_text = spark_rdd.map(lambda row: row['value'].split())


In [134]:
print(len(tweets_text.take(2)))
#tweets_text.take(2)

2


In [135]:
from pyspark.mllib.feature import HashingTF, IDF
hashingTF = HashingTF()

In [136]:
# Apply HashingTF to our DataFrame
tf = hashingTF.transform(tweets_text)
#
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [137]:
# Compute the TF-IDF
idfIgnore = IDF(minDocFreq=2).fit(tf)
tfidfIgnore = idfIgnore.transform(tf)

In [ ]:
# 1st value is the number of features
# 2nd list, features present in the document (each document is a line)
# 3rd list, cluster labels for each document

In [138]:
print("tfidf:")
for each in tfidf.collect():
    # Each vector is a tweet.
    print(each)

tfidf:
(1048576,[1475,105642,146549,154253,511771,685591,698511,734443,780680,840959,965976,973256,997716,1001303],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.22314355131420976,0.0,0.0,0.0])
(1048576,[1475,2209,3196,4437,6028,6421,7200,7357,8631,10099,10433,11012,11416,12743,14239,14394,15069,15153,15361,15774,16371,16538,17222,18887,18921,20318,21226,21309,21314,22217,22225,22308,23875,24009,24138,24895,25604,26018,26255,26759,27465,27527,27817,28088,30255,30758,31496,32134,32523,32922,32990,35930,36099,36647,38594,38649,39140,39436,40200,40267,40306,40649,41238,41843,42440,42603,43974,43983,45137,45574,45706,46357,47635,48829,48840,49323,50432,52138,52597,53191,53807,54245,54779,55207,55318,56375,56778,58084,58124,59798,59805,61050,61371,62073,63678,64512,64618,67697,68059,68131,69372,70882,71153,71506,72260,73059,73407,74402,75611,77506,77936,78129,78479,78681,79118,79172,80105,81278,81387,81569,84359,85045,85361,86920,87067,88065,88894,88908,89273,89573,90825,92613,93205,93284,93437

In [ ]:
from operator import add
tfidf_reduced = tfidf.reduceByKey(add)
tfidf_reduced.take(1)

In [51]:
print(tfidf)
tfidf_list = tfidf.collect()

MapPartitionsRDD[75] at mapPartitions at PythonMLLibAPI.scala:1340


In [ ]:
tfidf_list

In [ ]:
# Save first 1000 rows of print to a file
with open("/home/amaru-razerblade/tfidf_output.txt", "a") as myfile:
    for idx, row in enumerate(tfidf.collect()):
        print(idx)
        if idx < 1000:
            myfile.write(str(row)+"\n")
        else:
            break



# print("tfidfIgnore:")
# for each in tfidfIgnore.collect():
#     print(each)

In [11]:
spark.stop()

In [10]:
sc.stop()

In [8]:
from pyspark import SparkContext
# $example on$
from pyspark.mllib.feature import HashingTF, IDF
# $example off$

sc = SparkContext(appName="TFIDFExample123") 
documents = sc.textFile("gs://bigdata-general/kmeans_data.txt").map(lambda line: line.split(" "))


